In [53]:
import re
import json
import logging
import traceback
import numpy as np
import pandas as pd

import orjson

#import dask.dataframe as dd
#from dask.distributed import Client

In [54]:
logging.basicConfig(
    filename='../logs/app.log',
    filemode='w',
    format='%(asctime)s - %(levelname)s - %(message)s')

logger = logging.getLogger('data_analysis')

### Load

In [2]:
path = '../data/order_table_202208081749.csv'
df = pd.read_csv(path, sep=';', parse_dates=['created_at_irpf', 'created_at_loan'])

In [3]:
path = '../data/Dicionario_Grafia_Banco_SRF-v14.sav'
bank_df = pd.read_spss(path)

In [4]:
path = '../data/Dicionario_de_Agencias_e_Postos_Bancarios_com_Apelidos_v28.sav'
branch_df = pd.read_spss(path)

In [5]:
df.head()

,person_id,loan_id,irpf_id,created_at_irpf,created_at_loan,safra_created,product_code,state,rev,value,ordem
0,1792740,12811556.0,32876504,2022-03-22,2022-03-22,202203,PERSONAL,3.0,9,"{""rev"":9,""objType"":""IrpfPersonInfoT"",""personId...",1
1,15057615,NaN,33004278,2022-05-13,NaT,202205,NaN,NaN,10,"{""rev"":10,""objType"":""IrpfPersonInfoT"",""personI...",1
2,14936007,12813121.0,32879551,2022-03-24,2022-03-24,202203,PERSONAL,2.0,10,"{""rev"":10,""objType"":""IrpfPersonInfoT"",""personI...",1
3,2667826,12812575.0,32878530,2022-03-23,2022-03-23,202203,PERSONAL,3.0,6,"{""rev"":6,""objType"":""IrpfPersonInfoT"",""personId...",1
4,15273265,12841289.0,32958660,2022-04-30,2022-04-30,202204,PERSONAL,5.0,8,"{""rev"":8,""objType"":""IrpfPersonInfoT"",""personId...",1


In [6]:
branch_df.groupby(['Bank', 'Branch'])['BankName'].count().rename('count').reset_index().sort_values(by='count', ascending=False)

,Bank,Branch,count
0,001,0000,1
26930,275,0704,1
26923,275,0680,1
26924,275,0681,1
26925,275,0682,1
...,...,...,...
13463,041,0897,1
13464,041,0898,1
13465,041,0900,1
13466,041,0901,1


### Transformation

In [235]:
# Pad left side of branch with 0 to be 4 digits string
# cut right side of branch string to keep only 4 first digits in string

# Bank: convert from string name to string code of bank

def get_json_value(data, key='riskInfo'):
    try:
        data = json.loads(data)
    except:
        data = {}
    
    return data.get(key, {})


def get_bank_code(bank_name, df_bank: pd.DataFrame):
    try:
        bank_code = df_bank.loc[
            df_bank['BankName'] == bank_name.upper(),
            'Codigo_Banco'].item()
    except ValueError as ve:
        bank_code = '###'
    
    return bank_code

def tax_to_pay(text):
    if re.search('resultado encontrado: saldo inexistente', text.lower()):
        return 'no_balance'
    elif re.search('resultado encontrado: imposto a pagar', text.lower()):
        return 'tax_to_pay'
    elif (re.search('situação da restituição', text.lower()) or
            re.search('em fila de restituição', text.lower())):
        return 'refund'
    else:
        return 'error'


def get_full_text(risk_dict):
    year_list = list(risk_dict.keys())

    full_msg_dict = {}

    for year in year_list:
        data = risk_dict.get(year, {})

        if data['is_declaration']:
            full_msg_dict[year] = data.get('full_status_text', '')

    return full_msg_dict

def get_full_text_series(risk_dict):
    year_list = list(risk_dict.keys())

    person_lst = []
    year_lst = []
    text_lst = []
    state_lst = []


    for year in year_list:
        data = risk_dict.get(year, {})

        if data.get('is_declaration', False):
            person_lst = data.get('cpf', '')
            year_lst.append(year)
            txt = data.get('full_status_text', '')
            text_lst.append(txt)
            state_lst.append(tax_to_pay(txt.lower()))

    return person_lst, year_lst, text_lst, state_lst

def get_year_data(risk_dict):
    year_list = list(risk_dict.keys())
    
    bank_branch_dict = {}

    for year in year_list:
        data = risk_dict.get(year, {})

        if data.get('is_declaration', False):
            if 'bank' in data:
                bank_branch_dict[year] = {
                                            'cpf': str(data.get('cpf', '')),
                                            'bank': str(
                                                get_bank_code(
                                                    str(data.get('bank', '')),
                                                    bank_df)
                                                ).upper(),
                                            'branch': str(
                                                data.get('branch', '')
                                                )[:4].zfill(4),
                                            'state': tax_to_pay(
                                                data.get('full_status_text', '')
                                            )
                                            
                }
            else:
                bank_branch_dict[year] = {
                                            'cpf': str(data.get('cpf', '')),
                                            'state': tax_to_pay(
                                                data.get('full_status_text', '')
                                            )
                }

    return bank_branch_dict

def set_irpf_columns(s):
    risk_info = get_json_value(s)
    a, b, c, d = get_full_text_series(risk_info)
    
    ds = pd.DataFrame()
    ds['cpf'] = a
    ds['year'] = b
    ds['text'] = c
    ds['state'] = d

    return ds

In [124]:
def get_json_value(df: pd.DataFrame, col: str):
    '''Takes a pandas dataframe and a string column-name.
    Extracts json object from specified column in dataframe.
    Returns original dataframe joined with normalized json as columns.'''

    try:
        df = df.copy()
    except Exception as e:
        logger.debug(str(e))
        raise(e)

    try:
        data = pd.json_normalize(
            df[col].apply(
                orjson.loads), max_level=0)
    except KeyError as e:
        logger.debug(str(e))
        return df
    else:
        col_lst = data.columns.difference(df.columns)
        return df.join(data[col_lst])

def explode_dict_col(df, col='riskInfo'):
    '''Explodes risk_dict where each row is a
    tax report year.'''

    df = df.copy()

    df.loc[:, 'tax_report_data'] = (
        df[col].apply(
            lambda x: x.values()))

    df = df.apply(pd.Series.explode).reset_index(drop=True).copy()

    return df

def extract_value_dict(data: dict, key: str, default=np.nan):
    '''Function receives dictionary with key string
    and returns value. If default is provided, returns
    default value when key does not exist, otherwise returns nan.'''

    try:
        status = data.get(key, default)
    except AttributeError as e:
        logger.debug(str(e))
        raise(e)

    return status

def map_normalize_dict(df: pd.DataFrame, col: str, map:dict):
    '''Receives Pandas dataframe, column name and
    dictionary containing new column names as keys and dict
    keys as values. Normalizes dict column in dataframe and returns
    original dataframe with new columns.'''

    df = df.copy()

    for new_col_name, dict_key in map.items():
        df.loc[:, new_col_name] = df[col].apply(
            lambda x: extract_value_dict(
                data=x, key=dict_key))
                
    return df.fillna(value=np.nan)

In [189]:
regex_not_consulted = re.compile(r'(^\s*$|.*\bdata\sde\snascimento\sinformada.*'
                                r'está\sdive.*|^não\scoletado[.]?$|\bocorreu'
                                r'\suma\sinconsistência\s?[.]\s?saldo[:]'
                                r'\s(rama|0004).*|^\bocorreu\suma\sinconsistência'
                                r'\s?[.]$)', re.IGNORECASE)

regex_not_declared = re.compile(r'(^consta\sapresentação\sde\sdeclaração\sanual'
                                r'\sde\sisento\b|^apresentação\sda\sdeclaração\s'
                                r'como\sisento\b|^declaração\sconsta\scomo\sisento'
                                r'|^(sua\s)?declaração\sconsta\scomo\spedido\sde'
                                r'\sregularização\b|^sua\sdeclaração\snão\sconsta'
                                r'\sna\sbase\sde\sdados\b|^ainda\snão\sestá\sna'
                                r'\sbase\b)', re.IGNORECASE)

regex_tax_refund = re.compile(r'(^creditada[.]?$|.*situação\sda\srestituição[:]'
                                r'\screditada\b.*|.*\ba\sconsulta\sà\ssituação'
                                r'\sdessa\sdeclaração\ssomente\sserá\spermitida'
                                r'\spor\smeio\sdo\scódigo\sde\sacesso\b|^aguardando'
                                r'\sreagendamento\spelo\scontribuinte[.]?$|^devolvida'
                                r'\sà\sreceita\sfederal[,]\sem\srazão\sdo\snão\sresgate\b.*'
                                r'|^enviada\spara\scrédito\sno\sbanco[.]?$|^reagendada'
                                r'\spara\scrédito\sno\sbanco[.]?|^os\sdados\sda\sliberação'
                                r'\sde\ssua\srestituição\sestão\sdescritos\b.*|.*\bsua'
                                r'\sdeclaração\sestá\sna\sbase\sde\sdados\sda\sreceita\b.*'
                                r'|^está\sna\sbase[,]\sutilize\so\sextrato\sdo\sirpf[.]?$'
                                r'|^sua\sdeclaração\sjá\sfoi\sprocessada[.]?$)',
                                re.IGNORECASE)

In [188]:
regex_not_declared.findall('Declaração consta como pedido de regularização(PR)')

[('Declaração consta como pedido de regularização', '')]

In [14]:
cols = ['person_id', 'loan_id', 'irpf_id',
        'created_at_irpf', 'product_code',
        'state', 'rev', 'riskInfo']

df = get_json_value(df, 'value')
df = explode_dict_col(df[cols])
df = df.rename(columns={'riskInfo': 'year'})

df.head()

,person_id,loan_id,irpf_id,created_at_irpf,product_code,state,rev,year,tax_report_data
0,1792740,12811556.0,32876504,2022-03-22,PERSONAL,3.0,9,2012,"{'cpf': '86639277620', 'lot': '002', 'bank': '..."
1,1792740,12811556.0,32876504,2022-03-22,PERSONAL,3.0,9,2013,"{'cpf': '86639277620', 'data': [['Sua declaraç..."
2,1792740,12811556.0,32876504,2022-03-22,PERSONAL,3.0,9,2015,"{'cpf': '86639277620', 'data': [['Sua declaraç..."
3,1792740,12811556.0,32876504,2022-03-22,PERSONAL,3.0,9,2016,"{'cpf': '86639277620', 'data': [['Sua declaraç..."
4,1792740,12811556.0,32876504,2022-03-22,PERSONAL,3.0,9,2017,"{'cpf': '86639277620', 'data': [['Sua declaraç..."


In [125]:
col_key_map = {
    'is_declaration': 'is_declaration',
    'full_status_text': 'full_status_text',
    'bank': 'bank',
    'branch': 'branch'}

df = map_normalize_dict(df, 'tax_report_data', col_key_map)

In [134]:
df[(df['is_declaration']==False) & df['bank'].notnull()].tax_report_data.iloc[0]

{'cpf': '13792298716',
 'lot': '004',
 'bank': 'BANCO DO BRASIL S.A',
 'data': [['Os dados da liberação de sua restituição estão descritos abaixo:'],
  ['Banco:', 'BANCO DO BRASIL S.A'],
  ['Agência:', '0893'],
  ['Lote:', '004'],
  ['Disponível a partir de:', '31/08/2021'],
  ['Situação da Restituição:',
   'Enviada para crédito no banco. Para obter maiores informações sobre a situação da restitução, consulte o',
   'Meu Imposto de Renda (Extrato da DIRPF)',
   '.',
   'Caso a restituição não tenha sido creditada, ligue para a Central de Atendimento BB 4004-0001 (capitais), 0800-729-0001 (demais localidades) e 0800-729-0088 (deficientes auditivos) ou entre em contato com qualquer agência do Banco do Brasil S.A. para solicitar/reagendar o crédito. Também é possível solicitar/reagendar o crédito pelo Portal BB acessando o endereço',
   'https://www.bb.com.br/irpf',
   '.'],
  ['Ao consultar o andamento de sua restituição do Imposto sobre a Renda da Pessoa Física, confira  as oportunidad

In [121]:
df.isna().sum()

person_id                0
loan_id              14544
irpf_id                  0
created_at_irpf          0
product_code         14544
state                14544
rev                      0
year                     0
tax_report_data          0
is_declaration           0
full_status_text         0
bank                127004
branch              127119
status                   0
dtype: int64

In [62]:
a = False
int(a)

0

In [19]:
is_declaration(df.tax_report_data.loc[0])

bool

In [14]:
aux = pd.json_normalize(df['value'].apply(json.loads), max_level=0)

In [38]:
aux['risk_data'] = aux.riskInfo.apply(lambda x: x.values())
aux.apply(pd.Series.explode).rename(columns={'personId': 'person_id', 'riskInfo': 'year'}).reset_index(drop=True).risk_data.loc[2]

{'cpf': '86639277620',
 'data': [['Sua declaração já foi processada.',
   'Resultado encontrado: Saldo inexistente de imposto a pagar ou a restituir.'],
  ['Em Brasília - DF', '22/03/2022', '22:18'],
  ['versão 01.20180815']],
 'name': 'REGINALDO HUMBERTO DOS SANTOS',
 'year': 2015,
 'status': 'Sua declaração já foi processada.',
 'raw_data': '<!DOCTYPE HTML>\n<html>\n<head>\n<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>\n<title>Receita Federal do Brasil</title>\n<link href="//www.receita.fazenda.gov.br/tema/css/rfb.css" rel="stylesheet"/>\n<script data-main="//www.receita.fazenda.gov.br/tema/main-built.js" src="//www.receita.fazenda.gov.br/tema/require.js"></script>\n<script src="//www.receita.fazenda.gov.br/estatistica/estatistica.js"></script>\n<script language="javascript">\r\n\t\t\tfunction aoSair(evento) {\r\n\t\t\t\t//if (window.Event) {\r\n\t\t\t\t//\tdocument.captureEvents(Event.MOUSEUP);\r\n\t\t\t\t//}\r\n\t\t\t\t//alert(evento);\r\n\t\t\t}\r\n\t\t\t//a

In [17]:
aux['riskInfo'].apply(pd.Series).loc[0]['2012']

{'cpf': '86639277620',
 'lot': '002',
 'bank': 'ITAU UNIBANCO S.A.',
 'data': [['Os dados da liberação de sua restituição estão descritos abaixo:'],
  ['Banco:', 'ITAU UNIBANCO S.A.'],
  ['Agência:', '3751'],
  ['Lote:', '002'],
  ['Disponível a partir de:', '16/07/2012'],
  ['Situação da Restituição:',
   'Creditada',
   'Caso a restituição não tenha sido creditada, ligue para a Central de Atendimento BB 4004-0001 (capitais), 0800-729-0001 (demais localidades) e 0800-729-0088 (deficientes auditivos) ou entre em contato com qualquer agência do Banco do Brasil S.A. para solicitar/reagendar o crédito. Também é possível solicitar/reagendar o crédito pelo Portal BB acessando o endereço',
   'https://www.bb.com.br/irpf',
   '.'],
  ['Ao consultar o andamento de sua restituição do Imposto sobre a Renda da Pessoa Física, confira  as oportunidades que o',
   'Tesouro Direto',
   'traz.\xa0 Para o contribuinte que busca liquidez, segurança e rentabilidade, títulos públicos representam uma excel

In [349]:
json.loads(df['value'].loc[0])['riskInfo']

{'2012': {'cpf': '86639277620',
  'lot': '002',
  'bank': 'ITAU UNIBANCO S.A.',
  'data': [['Os dados da liberação de sua restituição estão descritos abaixo:'],
   ['Banco:', 'ITAU UNIBANCO S.A.'],
   ['Agência:', '3751'],
   ['Lote:', '002'],
   ['Disponível a partir de:', '16/07/2012'],
   ['Situação da Restituição:',
    'Creditada',
    'Caso a restituição não tenha sido creditada, ligue para a Central de Atendimento BB 4004-0001 (capitais), 0800-729-0001 (demais localidades) e 0800-729-0088 (deficientes auditivos) ou entre em contato com qualquer agência do Banco do Brasil S.A. para solicitar/reagendar o crédito. Também é possível solicitar/reagendar o crédito pelo Portal BB acessando o endereço',
    'https://www.bb.com.br/irpf',
    '.'],
   ['Ao consultar o andamento de sua restituição do Imposto sobre a Renda da Pessoa Física, confira  as oportunidades que o',
    'Tesouro Direto',
    'traz.\xa0 Para o contribuinte que busca liquidez, segurança e rentabilidade, títulos públic

In [298]:
json.loads(df['value'].loc[0])['riskInfo'].get('2019', {}).get('is_declaration', '')

True

In [304]:
df_test.text

0        Situação da Restituição: Creditada Caso a rest...
1        Sua declaração já foi processada. Resultado en...
2        Sua declaração já foi processada. Resultado en...
3        Sua declaração já foi processada. Resultado en...
4        Sua declaração já foi processada. Resultado en...
                               ...                        
51725    Sua declaração já foi processada. Resultado en...
51726    Sua declaração já foi processada. Resultado en...
51727    Sua declaração já foi processada. Resultado en...
51728    Sua declaração já foi processada. Resultado en...
51729    Sua declaração já foi processada. Resultado en...
Name: text, Length: 51730, dtype: object

In [236]:
df_test = pd.concat([set_irpf_columns(x) for x in df['value']], ignore_index=True)

In [ ]:
no_balance = 'resultado encontrado: saldo inexistente',
tax_to_pay = 'resultado encontrado: imposto a pagar',
refund =  'restituição'

In [323]:
no_balance = 'resultado encontrado: saldo inexistente'
tax_to_pay = 'resultado encontrado: imposto a pagar'
refund =  'restituição'

df_test['text'] = df_test['text'].str.lower()

df_test['no_balance'] = (
    np.where(df_test.text.str.contains(no_balance),
    1, 0))

df_test['tax_to_pay'] = (
    np.where(df_test.text.str.contains(tax_to_pay),
    1, 0))

df_test['refund'] = (
    np.where(df_test.text.str.contains(refund),
    1, 0))

In [335]:
df_test[df_test['state'] == 'refund'].text.value_counts()

situação da restituição: creditada caso a restituição não tenha sido creditada, ligue para a central de atendimento bb 4004-0001 (capitais), 0800-729-0001 (demais localidades) e 0800-729-0088 (deficientes auditivos) ou entre em contato com qualquer agência do banco do brasil s.a. para solicitar/reagendar o crédito. também é possível solicitar/reagendar o crédito pelo portal bb acessando o endereço https://www.bb.com.br/irpf .    33647
sua declaração está na base de dados da receita federal com a seguinte situação: processada - em fila de restituição.                                                                                                                                                                                                                                                                                                                               44
Name: text, dtype: int64

In [328]:
df_test.tax_to_pay.value_counts()

0    44271
1     7459
Name: tax_to_pay, dtype: int64

In [ ]:
## erro = não conseguiu coletar

In [313]:
df_test.text.str.match(txt_dic['no_balance'])

'resultado encontrado: saldo inexistente'

In [322]:
df_test['state_2'].value_counts()

refund        33691
no_balance    10493
tax_to_pay     7459
error            87
Name: state_2, dtype: int64

In [316]:
df_test['state_2'].value_counts()

refund        33691
no_balance    10493
tax_to_pay     7459
error            87
Name: state_2, dtype: int64

In [332]:
df_test[df_test['state'] == 'error']['text'].unique()

array(['sua declaração está na base de dados da receita federal.',
       'sua declaração já foi processada.'], dtype=object)

In [ ]:
## Está na base -> Vai receber a restituição e não sabe a agência
## Já foi declarada -> Vai receber a restituição

# Coloca na contagem da restituição, entra como banco comum
# Para tratar, considerar como agência comum e na contagem das restituições

# Base dos Status -> Pedir pra Jana. Receita pode criar um status novo.
# Trabalho 4 Life

# Cobrar da Jana.

# Admin Geru vai estar disponível em até 1 dia
# Treinamento da AWS no dia 30

In [289]:
df_test.cpf

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
51725    NaN
51726    NaN
51727    NaN
51728    NaN
51729    NaN
Name: cpf, Length: 51730, dtype: object

In [303]:
df_test[df_test['state'] == 'no_balance'].text.value_counts()

Sua declaração já foi processada. Resultado encontrado: Saldo inexistente de imposto a pagar ou a restituir.    10493
Name: text, dtype: int64

In [281]:
for i in to_pay_lst[260:270]:
    print (i)
    print('-----------------------')

In [144]:
texts = df['value'].apply(lambda x: get_full_text(get_json_value(x)))

In [181]:
texts.tax_to_pay

{'2017': 'Situação da Restituição: Creditada Caso a restituição não tenha sido creditada, ligue para a Central de Atendimento BB 4004-0001 (capitais), 0800-729-0001 (demais localidades) e 0800-729-0088 (deficientes auditivos) ou entre em contato com qualquer agência do Banco do Brasil S.A. para solicitar/reagendar o crédito. Também é possível solicitar/reagendar o crédito pelo Portal BB acessando o endereço https://www.bb.com.br/irpf .',
 '2018': 'Situação da Restituição: Creditada Caso a restituição não tenha sido creditada, ligue para a Central de Atendimento BB 4004-0001 (capitais), 0800-729-0001 (demais localidades) e 0800-729-0088 (deficientes auditivos) ou entre em contato com qualquer agência do Banco do Brasil S.A. para solicitar/reagendar o crédito. Também é possível solicitar/reagendar o crédito pelo Portal BB acessando o endereço https://www.bb.com.br/irpf .',
 '2019': 'Situação da Restituição: Creditada Caso a restituição não tenha sido creditada, ligue para a Central de At

In [81]:
df['value'].apply(lambda x: get_year_data(get_json_value(x)))

0        {'2012': {'bank': '341', 'branch': '3751'}, '2...
1              {'2016': {'bank': '104', 'branch': '0793'}}
2                                                       {}
3                                                       {}
4              {'2019': {'bank': '001', 'branch': '0893'}}
                               ...                        
18280    {'2014': {'bank': '104', 'branch': '1092'}, '2...
18281                                                   {}
18282                                                   {}
18283          {'2019': {'bank': '###', 'branch': 'None'}}
18284                                                   {}
Name: value, Length: 18285, dtype: object

In [71]:
list(json.loads(df['value'].loc[1])['riskInfo']['2016'].keys())

['cpf',
 'lot',
 'bank',
 'data',
 'name',
 'year',
 'branch',
 'status',
 'raw_data',
 'birthdate',
 'credit_date',
 'is_exemption',
 'is_declaration',
 'full_status_text']

In [340]:
json.loads(df['value'].loc[2])['riskInfo']['2013']

{'cpf': '42092321811',
 'data': [['Sua declaração não consta na base de dados da Receita Federal.'],
  ['Em Brasília - DF', '24/03/2022', '11:16'],
  ['versão 01.20180815']],
 'name': 'BELMIRO DE LIMA REZENDE',
 'year': 2013,
 'status': 'Sua declaração não consta na base de dados da Receita Federal.',
 'raw_data': '<!DOCTYPE HTML>\n<html>\n<head>\n<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>\n<title>Receita Federal do Brasil</title>\n<link href="//www.receita.fazenda.gov.br/tema/css/rfb.css" rel="stylesheet"/>\n<script data-main="//www.receita.fazenda.gov.br/tema/main-built.js" src="//www.receita.fazenda.gov.br/tema/require.js"></script>\n<script src="//www.receita.fazenda.gov.br/estatistica/estatistica.js"></script>\n<script language="javascript">\r\n\t\t\tfunction aoSair(evento) {\r\n\t\t\t\t//if (window.Event) {\r\n\t\t\t\t//\tdocument.captureEvents(Event.MOUSEUP);\r\n\t\t\t\t//}\r\n\t\t\t\t//alert(evento);\r\n\t\t\t}\r\n\t\t\t//alert(captureEvents(Event));\r

In [17]:
json.loads(df['value'].loc[0])['riskInfo']['2012']['is_declaration']

True

In [25]:
dicto = {}

for l2 in json.loads(df['value'].loc[0])['riskInfo']['2012']['data']:
    dicto[l2[0]] = ' '.join(l2[1:])

In [30]:
print(dicto['Banco:'])
print(dicto['Agência:'])

ITAU UNIBANCO S.A.
3751
